#Conversational

# Install requirements

In [ ]:
!pip install -q unsloth accelerate peft trl transformers bitsandbytes datasets

#Load Mistral Model with Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
import transformers

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",  # closest Unsloth-compatible Mistral variant
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

#Create a Small Chat Dataset (Conversational)

In [ ]:
from datasets import Dataset

chat_data = [
    {
        "instruction": "Hi there, how can I improve my sleep?",
        "input": "",
        "output": "Hello! Here are a few tips: maintain a consistent sleep schedule, avoid screens before bed, and create a calming bedtime routine."
    },
    {
        "instruction": "Can you explain what a transformer model is in simple terms?",
        "input": "",
        "output": "Sure! A transformer is a type of neural network that processes input data in parallel using attention mechanisms to understand relationships between words or tokens."
    },
    {
        "instruction": "What's a good way to learn Python as a beginner?",
        "input": "",
        "output": "Start with small projects, follow tutorials on websites like freeCodeCamp or W3Schools, and practice coding every day. Consistency is key!"
    },
]

dataset = Dataset.from_list(chat_data)

#Format Chat Prompts

In [ ]:
def format_chat(example):
    prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>
{example["instruction"]}{example["input"]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{example["output"]}<|eot_id|>"""
    example["text"] = prompt
    return example

dataset = dataset.map(format_chat)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

#Prepare LoRA for Training

In [ ]:
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

#Training Setup

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=30,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="mistral_chat_lora",
    save_strategy="no"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3 | Num Epochs = 30 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488/7,000,000,000 (0.19% trained)


Step,Training Loss
1,1.934700
2,1.934800
3,1.702400
4,1.424500
5,1.115600
6,0.828500
7,0.579900
8,0.409900
9,0.286500
10,0.188900


TrainOutput(global_step=30, training_loss=0.39929490561286607, metrics={'train_runtime': 56.0307, 'train_samples_per_second': 4.283, 'train_steps_per_second': 0.535, 'total_flos': 414847498936320.0, 'train_loss': 0.39929490561286607})

#Inference Block (Clean)

In [ ]:
prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Can you tell me how to stay productive when working from home?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

eos_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>") or tokenizer.eos_token_id

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.5,
        top_p=0.9,
        do_sample=True,
        eos_token_id=eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

# Extract the assistant's response
if "<|start_header_id|>assistant<|end_header_id|>" in decoded:
    response = decoded.split("<|start_header_id|>assistant<|end_header_id|>")[1]
    print("=== Assistant Response ===\n")
    print(response.split("<|")[0].strip())
else:
    print("⚠️ Could not find assistant response. Raw output:\n", decoded)


=== Assistant Response ===

Sure! Here are a few tips: maintain a routine, take breaks, and avoid distractions.
